# 第5回講義 宿題

## 課題. Tensorflowを用いて, MNISTを多層パーセプトロン(MLP)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

- CNNは使わないでください

**`tf` の以下のモジュールはこの回では使用できないように制限されています. 注意してください.**
```python
tf.app
tf.compat
tf.contrib
tf.erros
tf.gfile
tf.graph_util
tf.image
tf.layers
tf.logging
tf.losses
tf.metrics
tf.python_io
tf.resource_loader
tf.saved_model
tf.sdca
tf.sets
tf.summary
tf.sysconfig
tf.test
tf.train
```

次のセルのhomework関数を完成させて提出してください

# Answer Cell

In [ ]:
def homework(train_X, train_y, test_X):
    
    def oneHotEnding(train_y):
        y_size = len(train_y)
        y = np.zeros((y_size, 10))
        y[np.arange(y_size), train_y] = 1
        return y
    
    # Placeholders
    x = tf.placeholder(tf.float32, [None, 784])
    t = tf.placeholder(tf.float32, [None, 10])
    # var
    eps = tf.Variable(0.01, name='eps')
    W1 = tf.Variable(np.random.uniform(low=-0.08, high=0.08, size=(784, 200)).astype('float32'), name='W1')
    b1 = tf.Variable(np.zeros(200).astype('float32'), name='b1')
    W2 = tf.Variable(np.random.uniform(low=-0.08, high=0.08, size=(200, 10)).astype('float32'), name='W2')
    b2 = tf.Variable(np.zeros(10).astype('float32'), name='b2')
    params = [W1, b1, W2, b2]
    
    # define graph
    u1 = tf.matmul(x, W1) + b1
    z1 = tf.nn.sigmoid(u1)
    u2 = tf.matmul(z1, W2) + b2
    y = tf.nn.softmax(u2)
    
    # define loss function
    cost = -tf.reduce_mean(tf.reduce_sum(t*tf.log(tf.clip_by_value(y, 1e-10, 1.0)), axis=1))
    
    # define update rule
    gW1, gb1, gW2, gb2 = tf.gradients(cost, params)
    updates = [
        W1.assign_sub(eps*gW1),
        b1.assign_sub(eps*gb1),
        W2.assign_sub(eps*gW2),
        b2.assign_sub(eps*gb2)
    ]
    train = tf.group(*updates)
    valid = tf.argmax(y, 1)
    
    # main
    # initial
    n_epochs = 1000
    batch_size = 100
    n_batches = train_X.shape[0] // batch_size
    # OneHotEncoding
    train_y = oneHotEnding(train_y)
    
    # learning procedure
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            train_X, train_y = shuffle(train_X, train_y, random_state=42)
            for i in range(n_batches):
                start = i * batch_size
                end = start + batch_size
                sess.run(train, feed_dict={x: train_X[start:end], t: train_y[start:end]})

    # get predictor answer
        pred_y = sess.run(valid, feed_dict={x: test_X})
    return pred_y

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 提出に際して、以下のscore_homework関数で60分で実行が終わることを確認して下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

# Checker Cell (for student)

In [ ]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

del [
    tf.app,
    tf.compat,
    tf.contrib,
    tf.errors,
    tf.gfile,
    tf.graph_util,
    tf.image,
    tf.layers,
    tf.logging,
    tf.losses,
    tf.metrics,
    tf.python_io,
    tf.resource_loader,
    tf.saved_model,
    tf.sdca,
    tf.sets,
    tf.summary,
    tf.sysconfig,
    tf.test,
    tf.train
]

def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X, mnist_y = shuffle(mnist.data.astype('float32'),
                               mnist.target.astype('int32'), random_state=42)

    mnist_X = mnist_X / 255.0

    return train_test_split(mnist_X, mnist_y,
                test_size=0.2,
                random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()

    # validate for small dataset
    train_X_mini = train_X[:100]
    train_y_mini = train_y[:100]
    test_X_mini = test_X[:100]
    test_y_mini = test_y[:100]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(test_y_mini, pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(test_y, pred_y, average='macro'))

In [ ]:
validate_homework()
#score_homework()